# 🧩 Bloque 0 — Setup inicial

En este notebook se extraen las **características de color, textura y forma**
de las imágenes de hojas de papa, que servirán como entrada para el modelo de Machine Learning.  

**Objetivo:** transformar las imágenes en vectores numéricos que describan sus propiedades visuales.


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.measure import label as sk_label, regionprops, moments_hu
from scipy.stats import skew, kurtosis
from tqdm import tqdm

## 🧠 Bloque 1 — Parámetros globales

En este bloque se definen las **rutas base** del proyecto y los parámetros generales que controlan el flujo de procesamiento.  
Las variables establecen:

- La carpeta donde se encuentran las imágenes ya redimensionadas (`base_path`).  
- La ruta donde se guardará el dataset resultante con las características extraídas (`output_csv`).  

Estos parámetros permiten mantener una estructura organizada y reutilizable para las siguientes etapas del análisis.


In [2]:
base_path = "../data/2_data_resize"
output_csv = "../data/3_data_extract_features/features_dataset.csv"

In [3]:
def extract_features_full(image):
    # -------------------- COLOR --------------------
    if image.shape[2] == 4:
        image = image[:, :, :3]
    
    mask = np.any(image > 15, axis=2)
    if np.sum(mask) < 100:
        mask = np.ones(mask.shape, dtype=bool)
    
    pixels = image[mask]
    mean_colors = np.mean(pixels, axis=0)
    std_colors = np.std(pixels, axis=0)
    skew_colors = skew(pixels, axis=0)
    kurt_colors = kurtosis(pixels, axis=0)
    
    hist_R, _ = np.histogram(pixels[:,2], bins=16, range=(0,255), density=True)
    hist_G, _ = np.histogram(pixels[:,1], bins=16, range=(0,255), density=True)
    hist_B, _ = np.histogram(pixels[:,0], bins=16, range=(0,255), density=True)
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_pixels = hsv[mask]
    mean_hsv = np.mean(hsv_pixels, axis=0)
    std_hsv = np.std(hsv_pixels, axis=0)
    
    R, G, B = mean_colors[2], mean_colors[1], mean_colors[0]
    ratio_RG = G/(R+1e-5)
    ratio_GB = G/(B+1e-5)
    ratio_RB = R/(B+1e-5)
    
    color_features = list(mean_colors) + list(std_colors) + list(skew_colors) + list(kurt_colors) + \
                     list(mean_hsv) + list(std_hsv) + [ratio_RG, ratio_GB, ratio_RB] + \
                     list(hist_R) + list(hist_G) + list(hist_B)
    
    # -------------------- TEXTURA --------------------
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_norm = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    lbp_features = []
    for radius in [1,2,3]:
        P = 8*radius
        lbp = local_binary_pattern(gray_norm, P=P, R=radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0,P+3), density=True)
        lbp_features.extend(hist)
    
    distances = [1,2,3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(gray_norm, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_features = [graycoprops(glcm, prop).mean() for prop in ['contrast','homogeneity','energy','correlation','dissimilarity','ASM']]
    entropy = -np.sum(glcm[:,:,0,0]*np.log2(glcm[:,:,0,0]+1e-10))
    
    texture_features = lbp_features + glcm_features + [entropy]
    
    # -------------------- FORMA --------------------
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    labeled = sk_label(thresh)
    regions = regionprops(labeled)
    if len(regions) == 0:
        shape_features = [0]*20
    else:
        region = max(regions, key=lambda r: r.area)
        area = region.area
        perimeter = region.perimeter if region.perimeter != 0 else 1e-5
        aspect_ratio = region.bbox[3]/(region.bbox[2]+1e-5)
        circularity = 4*np.pi*area/(perimeter**2)
        solidity = region.solidity
        extent = region.extent
        hu = moments_hu(region.image)
        shape_features = [area, perimeter, aspect_ratio, circularity, solidity, extent] + list(hu)
    
    return color_features + texture_features + shape_features

## 🧾 Bloque 4 — Extracción global por carpeta

En este bloque se **recorren todas las subcarpetas del dataset**, donde cada carpeta representa una clase (por ejemplo, diferentes tipos o estados de hojas).

Para cada imagen:
1. Se **lee y convierte** a escala de grises.  
2. Se **extraen las características de color y textura** mediante las funciones definidas previamente.  
3. Se **almacenan los valores estadísticos** (promedios, desviaciones y métricas de textura) junto con su **etiqueta de clase**.

El resultado es un **DataFrame estructurado**, donde cada fila corresponde a una imagen y cada columna a una característica relevante para su posterior análisis o clasificación.


In [4]:
data_rows = []

for label_name in os.listdir(base_path):
    class_path = os.path.join(base_path, label_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Procesando {label_name}"):
        img_path = os.path.join(class_path, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        features = extract_features_full(image)
        row = {f"f{i}": features[i] for i in range(len(features))}
        row["label"] = label_name
        data_rows.append(row)

Procesando Bacteria:   0%|          | 1/2276 [00:00<37:33,  1.01it/s]C:\Users\Usuario\AppData\Local\Temp\ipykernel_20596\1810162350.py:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_colors = skew(pixels, axis=0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20596\1810162350.py:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt_colors = kurtosis(pixels, axis=0)
Procesando Virus: 100%|██████████| 2128/2128 [12:01<00:00,  2.95it/s]


## 💾 Bloque 5 — Ejecución y guardado

En este bloque se **consolidan todas las características extraídas** en un `DataFrame` de *pandas* y se **exportan a un archivo CSV**.

Este dataset servirá como **entrada para los siguientes procesos** de análisis, visualización o clasificación con Machine Learning.

Cada fila del archivo contiene:
- Las **características de color** (medias y desviaciones por canal).  
- Las **características de textura** (contraste, homogeneidad, energía y entropía).  
- La **etiqueta** correspondiente al grupo o clase (`label`).

El CSV generado puede integrarse directamente en flujos de *data preprocessing* o modelos de aprendizaje supervisado.


In [5]:
df = pd.DataFrame(data_rows)
df.to_csv(output_csv, index=False)
print(f"Dataset guardado en: {output_csv}")
print(df.head())

Dataset guardado en: ../data/3_data_extract_features/features_dataset.csv
           f0          f1          f2         f3         f4         f5  \
0   66.608757  118.838588   95.380712  50.503801  54.388886  56.084155   
1  255.000000  255.000000  255.000000   0.000000   0.000000   0.000000   
2   65.542403  119.060494   95.172101  50.529366  54.456675  56.236278   
3   68.384251   68.384251   68.384251  43.681501  43.681501  43.681501   
4   65.752993  139.603055  125.398704  53.922779  64.747192  65.821055   

         f6        f7        f8        f9  ...      f134      f135  f136  \
0  0.709024  0.264551  0.408486 -0.357151  ...  0.480139  0.356773   0.0   
1       NaN       NaN       NaN       NaN  ...  0.130702  0.080689   0.0   
2  0.817188  0.240932  0.425553 -0.119683  ...  0.480139  0.356773   0.0   
3  1.035692  1.035692  1.035692  0.574811  ...  0.373599  0.277572   0.0   
4  0.450341  0.022884  0.106899 -0.855384  ...  0.786730  0.744599   2.0   

   f137  f138  f139  f14